# The Graph Module 

`graph` is a module (written in python) to effectively manipulate data structured as a graph. The basic unit is the `Graph` class, which stores the data of the graph (both the topology and any properties/labels the nodes and edges might carry), and comes equipped with a minimal but sufficient set of methods which enable any natural graph computation. The full [API](https://davidlibland.github.io/graph-module) can be found at: https://davidlibland.github.io/graph-module

In choosing the methods for the `Graph` class, the fundamental design decision we made was that computations should be **local** and **parallel** with respect to the graph structure, so that they could be easily distributed. Thus, the major methods:

1. `Graph.new_projection`
2. `Graph.new_subgraph`
3. `Graph.update_edges`
4. `Graph.update_nodes`
5. `Graph.send_collect`

Each take a node (and/or edge) function, which are then applied in parallel to each node (and/or edge).

The first two methods:

* `Graph.new_projection`
* `Graph.new_subgraph`

return new graphs (possibly with new topologies), and leave the previous graph unaltered. The next two methods:
* `Graph.update_edges`
* `Graph.update_nodes`

change the state of the nodes and edges. All of these methods behave in an essentially pointwise manner - they act on each node/edge in isolation of the others.

The final method:
* `Graph.send_collect`

enables local communication within the graph: Edges are given instructions on which messages to *send* to their source and destination nodes, and then nodes are given instructions on how to *collect* and process those messages. Details can be found in the [API](https://davidlibland.github.io/graph-module). 


Finally, there are methods to add new nodes and edges (`Graph.add_nodes`, and `Graph.add_edges`), and methods which allow one to query the graph data in a structurally relevant way (`Graph.nodes` and `Graph.find`).

## Examples:

In [1]:
from graph import Graph

First we create a new graph. Edges are represented as triples:

(source node object, destination node object, edge label object)

for instance `('A','B',1)` indicates an edge from `'A'` to `'B'`, labeled by `1`.

In [24]:
g = Graph(edges = [('A','B',1),('A','C',2),
                   ('B','C',2),('C','D',1),('D','A',3)],
          nodes = {'E'})
g

Nodes:
    1 'D'
    2 'E'
    3 'A'
    4 'C'
    5 'B'

Edges:
     Source Node              Edge Object              Destination Node
    1'A'                      1.0                      'B'                     
    2'A'                      2.0                      'C'                     
    3'B'                      2.0                      'C'                     
    4'C'                      1.0                      'D'                     
    5'D'                      3.0                      'A'                     

`g` would look like this:
![](imgs/graph1.png)

We could then query the graph, looking for cyclic love triangles:

In [3]:
g.find('(x)-[]->(y); (y)-[]->(z); (z)-[]->(x)')

,(x),(y),(z)
0,A,C,D
1,C,D,A
2,D,A,C


There are three of them (up to rotation), but there is only one directed love triangle:

In [4]:
g.find('(x)-[]->(y); (y)-[]->(z); (x)-[]->(z)')

,(x),(y),(z)
0,A,B,C


## Graph Algorithm Examples

More complicated algorithms will require the nodes edge objects to communicate with each other and react accordingly (and often record intermediate states). In particular, the objects attached to the nodes and edges cannot be immutable. The module provides a convenient dict-like class to wrap over any object (the `GraphLabel` class) but it isn't necessary to use it.

In [5]:
from graph.algorithms import GraphLabel

We can wrap all the node and edge objects in our previous graph using the `new_projection` method.

In [6]:
g_mutable = g.new_projection(GraphLabel.edge_wrapper,GraphLabel.node_wrapper)
g_mutable

Nodes:
    1 'A' : {'data': 'A'}
    2 'E' : {'data': 'E'}
    3 'D' : {'data': 'D'}
    4 'C' : {'data': 'C'}
    5 'B' : {'data': 'B'}

Edges:
     Source Node              Edge Object              Destination Node
    1'A' : {'data': 'A'}      '1.0' : {'data': 1.0}    'B' : {'data': 'B'}     
    2'A' : {'data': 'A'}      '1.0' : {'data': 1.0}    'C' : {'data': 'C'}     
    3'B' : {'data': 'B'}      '2.0' : {'data': 2.0}    'C' : {'data': 'C'}     
    4'C' : {'data': 'C'}      '2.0' : {'data': 2.0}    'D' : {'data': 'D'}     
    5'D' : {'data': 'D'}      '3.0' : {'data': 3.0}    'A' : {'data': 'A'}     

Let's write a simple algorithm, `out_degree` which counts the number of edges leaving each node, and stores the result as an attribute for the node:

In [7]:
def out_degree(g):
    # Each edge emits a non-empty message to the source node (and an empty message to the destination node)
    def emitter(src,dst,e):
        return [1],[]
    
    # Each node collects all the messages it receives and counts them, the result is the number of outgoing edges.
    def collector(node,msgs):
        node['out_degree'] = len(list(msgs))
    
    g.send_collect(emitter,collector)

In [8]:
out_degree(g_mutable)
g_mutable

Nodes:
    1 'A' : {'data': 'A', 'out_degree': 2}
    2 'E' : {'data': 'E', 'out_degree': 0}
    3 'D' : {'data': 'D', 'out_degree': 1}
    4 'C' : {'data': 'C', 'out_degree': 1}
    5 'B' : {'data': 'B', 'out_degree': 1}

Edges:
     Source Node              Edge Object              Destination Node
    1'A' : {'data': 'A',  ...  '1.0' : {'data': 1.0}    'B' : {'data': 'B',  ... 
    2'A' : {'data': 'A',  ...  '1.0' : {'data': 1.0}    'C' : {'data': 'C',  ... 
    3'B' : {'data': 'B',  ...  '2.0' : {'data': 2.0}    'C' : {'data': 'C',  ... 
    4'C' : {'data': 'C',  ...  '2.0' : {'data': 2.0}    'D' : {'data': 'D',  ... 
    5'D' : {'data': 'D',  ...  '3.0' : {'data': 3.0}    'A' : {'data': 'A',  ... 

## Page Rank

We now write a slightly more complicated algorithm, PageRank:

In [9]:
def page_rank(g,reset_prob,threshold=0.001):
    if not (reset_prob >= 0  and reset_prob <=1):
        raise ValueError('We require 0 <= reset_prob <= 1')
        
    if not threshold > 0:
        raise ValueError('We require 0 < threshold')
        
    #Initialize the nodes and edges:
    out_degree(g) # make sure each node knows it's out_degree
    
    #Each node starts the algorithm with a PageRank of 1.
    def init_node(node):
        node['page_rank'] = 1.
        node['halt'] = False
    g.update_nodes(init_node)
    
    #Each edge remember the proportion of the traffic if carries out of it's source:
    def init_edge(src,dst,e):
        e['traffic_prop'] = 1./src['out_degree']
    g.update_edges(init_edge)
    
    #The emitter sends the relevant proportion of the 
    #source node's page_rank along each edge:
    def emitter(src,dst,e):
        return [],[src['page_rank']*e['traffic_prop']]
        
    

    #Dangling nodes (nodes with out_degree==0) cannot transmit it's current page_rank
    #along any outgoing edges, so we need to collect their current page_ranks centrally
    #and redistibute them equally amongst all nodes.
    num_nodes = sum(1 for node in g.nodes())
    
    node_pred = lambda node : node['out_degree']==0
    dangling_nodes = g.new_subgraph(node_pred=node_pred).nodes()
    
    def page_rank_collector(node,incoming_ranks,avg_dangle_rank):
        old_rank = node['page_rank']
        node['page_rank'] = (1-reset_prob)*(sum(incoming_ranks)+avg_dangle_rank)+reset_prob  
        if abs(old_rank - node['page_rank']) < threshold:
            node['halt']=True
        else:
            node['halt']=False
    
    while sum( 1 for node in g.nodes() if not node['halt']) > 0:
        avg_dangle_rank = sum(node['page_rank'] for node in dangling_nodes)/num_nodes
        collector = lambda node,msgs: page_rank_collector(node,msgs,avg_dangle_rank)
        g.send_collect(emitter,collector)

Let's compute it on the graph `g_mutable`

In [19]:
page_rank(g_mutable,0.15)

#Print the output:
print('Node:     PageRank:')
for node in g_mutable.nodes():
    print("{:<10}".format(node.name) + str(node['page_rank']))

Node:     PageRank:
A         1.3332985925934921
E         0.18072289156626506
D         1.355970523917598
C         1.3826372423498343
B         0.7473707495728084


We can visualize it as:
![](imgs/page_rank.png)

## Discussion

#### Page Rank

The are various ways in which PageRank (or variants of it), might be useful to Human Dx. As some examples, one could use a variants of it
* to recommend cases to experts 
* to recommend paedogogically relevant cases to students
* given a subset of information about a case, it could be used to try and predict which pieces of information (test results/family history) experts are most likely to inquire about next. I.e. what indicators are seen as most relevant to a diagnosis.
* to predict potential links between symptoms and conditions (given a sparse graph of known links)


#### Scalability of the Graph API

The API itself is relatively scalable; but one would need to use scalable data structures. Some thoughts:
* To facilitate partitioning the data and parallelizing the computations, all node objects should be of the same class, and similarily, all edge objects should be of the same class.
* It could save space and computational resources if the computations in a graph algorithm were executed in a lazy manner: Nodes and edges will collect a list of instructions (send, process, and update requests), but only the instructions relevant to the query will be executed.

#### Further discussion

**Undirected graphs**
* We don't implement undirected graphs directly. However, the graph methods do not inherantly treat the source or destination nodes any differently, so undirected graph algorithms can be written easily. For example the following algorithm computes the connected components of the (directionless) graph:

In [21]:
def connected_comp(g):
    # We initialize each node by setting node['cc'] to it's own id.
    def init_node(node):
        node['cc'] = node.name+" id:"+str(id(node))
        node['halt'] = True
    g.update_nodes(init_node) #Initialize vertices 
    
    # Each edge sends messages to both nodes informing them of the value of each other's node['cc']
    def emitter(src,dst,e):
        return [dst['cc']],[src['cc']]

    # The node sets node['cc'] to the smallest value received from it's neighbors.
    def collector(node,msg_iter):
        node['halt'] = True
        for near_cc in msg_iter:
            if near_cc < node['cc']:
                node['cc'] = near_cc
                node['halt'] = False
    
    g.send_collect(emitter,collector)
    while sum( 1 for node in g.nodes() if not node['halt'])>0:
        g.send_collect(emitter,collector)

In [25]:
connected_comp(g_mutable)

#Print the output:
print('Node:     Component:')
for node in g_mutable.nodes():
    print("{:<10}".format(node.name) + str(node['cc']))

Node:     Component:
A         A id:4491870728
E         E id:4491872408
D         A id:4491870728
C         A id:4491870728
B         A id:4491870728


**Additional Algorithms**
There are relatively good distributed algorithms for Breadth First Search, shortest path, span, various notions of centrality, and variations of clustering, among others. All of these can be implemented using the API. On the other hand, some basic graph algorithms such as Depth First Search are inherently sequential, and cannot be implemented in a distributed way.